# 3.1 

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [4]:
# Parameters
gamma = 1  # Discount factor
alpha = 0.1  # Learning rate
iterations = 500
col_size = 21
row_size = 4
epsilon = 0.1

In [5]:
# Grid and states
cliff = [(3, i) for i in range(1, col_size - 1)]
start = (3, 0)
goal = (3, 20)
states = [(i, j) for i in range(row_size) for j in range(col_size)]

In [6]:
# Actions
actions = [(-1, 0), (1, 0), (0, 1), (0, -1)]

def transition(state, action):
    new_state = (max(min(state[0] + action[0], row_size - 1), 0),
                 max(min(state[1] + action[1], col_size - 1), 0))
    return new_state if new_state != state else state

def take_action(state, action):
    next_state = transition(state, action)
    reward = 20 if next_state == goal else -100 if next_state in cliff else -1
    terminal = next_state == goal or next_state in cliff
    return next_state, reward, terminal

def choose_action(state, qvalues, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.choice(actions)
    else:
        return max(actions, key=lambda a: qvalues[state][a])

def show_route(visited_states):
    for i in range(row_size):
        print('-' * (col_size * 4))
        for j in range(col_size):
            token = 'R' if (i, j) in visited_states else 'G' if (i, j) == goal else '*' if (i, j) in cliff else '0'
            print(f'| {token} ', end='')
        print('|')
    print('-' * (col_size * 4))

In [7]:
def showRoute(states):
    board = np.zeros([row_size, col_size])
    # Add cliff marked as -1
    for c in cliff:
        board[c[0], c[1]] = -1

    for state in states:
        if state != start and state != goal:  # Avoid overwriting start and goal
            board[state[0], state[1]] = 1  # Mark the route

    for i in range(row_size):
        print('--------------------------------------------')
        out = ' '
        for j in range(col_size):
            token = ' '
            if board[i, j] == -1:
                token = 'C'
            elif board[i, j] == 1:
                token = '1'
            if (i, j) == start:
                token = 'S'
            if (i, j) == goal:
                token = 'G'
            out += token + ' '
        print(out)
    print('--------------------------------------------')

In [8]:
def sarsa(episodes, epsilon, start, alpha, qvalues=None):
    if qvalues is None:
        qvalues = {(i, j): {tuple(a): 0.0 for a in actions}  # Convert actions to tuples
                   for i in range(row_size) for j in range(col_size)}

    for episode in range(episodes):
        current_state = start
        current_action = choose_action(current_state, qvalues, epsilon)
        terminal = False

        while not terminal:
            next_state, reward, terminal = take_action(current_state, current_action)
            next_action = choose_action(next_state, qvalues, epsilon) if not terminal else None

            # SARSA update
            current_value = qvalues[current_state][current_action]
            next_value = qvalues[next_state][next_action] if next_action is not None else 0
            qvalues[current_state][current_action] += alpha * (reward + gamma * next_value - current_value)

            current_state, current_action = next_state, next_action

    return qvalues

In [9]:
def qlearning(episodes, epsilon, start, alpha, gamma, qvalues=None):
    if qvalues is None:
        qvalues = {(i, j): {a: 0.0 for a in actions}
                   for i in range(row_size) for j in range(col_size)}

    for episode in range(episodes):
        current_state = start
        terminal = False

        while not terminal:
            current_action = choose_action(current_state, qvalues, epsilon)
            next_state, reward, terminal = take_action(current_state, current_action)

            # Q-Learning update
            current_value = qvalues[current_state][current_action]
            next_value = max(qvalues[next_state].values()) if not terminal else 0
            qvalues[current_state][current_action] += alpha * (reward + gamma * next_value - current_value)

            current_state = next_state

    return qvalues

In [10]:
def generate_policy_path(start, qvalues):
    current_state = start
    policy_path = [current_state]
    terminal = False

    while not terminal:
        current_action = max(qvalues[current_state], key=qvalues[current_state].get)
        next_state, _, terminal = take_action(current_state, current_action)
        policy_path.append(next_state)
        current_state = next_state
        if current_state == goal:  
            break

    return policy_path

In [21]:
# Run SARSA to get Q-values
qs_s = sarsa(1000, epsilon, start, alpha)

# Generate policy path from the learned Q-values
states_s = generate_policy_path(start, qs_s)

print(states_s)
showRoute(states_s)

[(3, 0), (2, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (2, 20), (3, 20)]
--------------------------------------------
                                           
--------------------------------------------
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
--------------------------------------------
 1                                       1 
--------------------------------------------
 S C C C C C C C C C C C C C C C C C C C G 
--------------------------------------------


In [22]:
# Run Q-learning to get Q-values
qs_q = qlearning(1000, epsilon, start, alpha, gamma)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

[(3, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (3, 20)]
--------------------------------------------
                                           
--------------------------------------------
                                           
--------------------------------------------
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
--------------------------------------------
 S C C C C C C C C C C C C C C C C C C C G 
--------------------------------------------


# 3.2

## Policies

In [14]:
#Test different epsilon value for SARSA
iterations = 500

print("e = 0")
epsilon = 0
qs_q = sarsa(1000, epsilon, start, alpha)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

print("e = 0.2")
epsilon = 0.2
qs_q = sarsa(1000, epsilon, start, alpha)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

print("e = 0.5")
epsilon = 0.5
qs_q = sarsa(1000, epsilon, start, alpha)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)


print("e = 0.8")
epsilon = 0.8
qs_q = sarsa(1000, epsilon, start, alpha)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

print("e = 1")
epsilon = 1
qs_q = sarsa(1000, epsilon, start, alpha)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

e = 0
[(3, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (3, 20)]
--------------------------------------------
                                           
--------------------------------------------
                                           
--------------------------------------------
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
--------------------------------------------
 S C C C C C C C C C C C C C C C C C C C G 
--------------------------------------------
e = 0.2
[(3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (2, 20), (3, 20)]
--------------------------------------------
 1 1 1 1 1 1 1 1 1 1 1 1 1 1               
--------------------------------------------
 1                     

KeyboardInterrupt: 

In [16]:
#Test different epsilon value for Q-Learning
iterations = 500

print("e = 0")
epsilon = 0
qs_q = qlearning(1000, epsilon, start, alpha, gamma)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

print("e = 0.2")
epsilon = 0.2
qs_q = qlearning(1000, epsilon, start, alpha, gamma)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

print("e = 0.5")
epsilon = 0.5
qs_q = qlearning(1000, epsilon, start, alpha, gamma)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

print("e = 0.8")
epsilon = 0.8
qs_q = qlearning(1000, epsilon, start, alpha, gamma)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

print("e = 1")
epsilon = 1
qs_q = qlearning(1000, epsilon, start, alpha, gamma)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

e = 0
[(3, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (3, 20)]
--------------------------------------------
                                           
--------------------------------------------
                                           
--------------------------------------------
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
--------------------------------------------
 S C C C C C C C C C C C C C C C C C C C G 
--------------------------------------------
e = 0.2
[(3, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (3, 20)]
--------------------------------------------
                                           
--------------------------------------------
                                           
-------------

MemoryError: 

## Rewards

# 3.3 Snake Pits Problem

In [57]:
# Parameters
gamma = 1  # Discount factor
alpha = 0.1  # Learning rate
iterations = 500
col_size = 21
row_size = 4
epsilon = 0.1

In [58]:
# Grid and states
cliff = [(3, i) for i in range(1, col_size - 1)]  
snake_pit = [(1, 10), (2, 10)]  # Snake pit locations in the two bottom cells of the 11th column
start = (3, 0)  
goal = (3, col_size - 1)  
states = [(i, j) for i in range(row_size) for j in range(col_size)]

In [59]:
# Actions
actions = [(-1, 0), (1, 0), (0, 1), (0, -1)]

def transition(state, action):
    new_state = (max(min(state[0] + action[0], row_size - 1), 0),
                 max(min(state[1] + action[1], col_size - 1), 0))
    return new_state if new_state != state else state

def take_action(state, action):
    next_state = transition(state, action)
    reward = 20 if next_state == goal else -100 if next_state in cliff or next_state in snake_pit else -1
    terminal = next_state == goal or next_state in cliff
    return next_state, reward, terminal

def choose_action(state, qvalues, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.choice(actions)
    else:
        return max(actions, key=lambda a: qvalues[state][a])

def show_route(visited_states):
    for i in range(row_size):
        print('-' * (col_size * 4))
        for j in range(col_size):
            token = 'R' if (i, j) in visited_states else 'G' if (i, j) == goal else '*' if (i, j) in cliff else '0'
            print(f'| {token} ', end='')
        print('|')
    print('-' * (col_size * 4))

In [60]:
def showRoute(states):
    board = np.zeros([row_size, col_size])

    # Mark cliff locations as -1
    for c in cliff:
        board[c[0], c[1]] = -1

    # Mark snake pit locations as -2
    for s in snake_pit:
        board[s[0], s[1]] = -2

    # Mark the route taken
    for state in states:
        if state != start and state != goal and board[state[0], state[1]] == 0:
            board[state[0], state[1]] = 1

    for i in range(row_size):
        print('--------------------------------------------')
        out = ''
        for j in range(col_size):
            token = ' '
            if board[i, j] == -1:
                token = 'C'  # Cliff
            elif board[i, j] == -2:
                token = 'O'  # Snake pit
            elif board[i, j] == 1:
                token = 'R'  # Route
            if (i, j) == start:
                token = 'S'  # Start
            if (i, j) == goal:
                token = 'G'  # Goal
            out += token + ' '
        print(out)
    print('--------------------------------------------')


In [61]:
def sarsa(episodes, epsilon, start, alpha, qvalues=None):
    if qvalues is None:
        qvalues = {(i, j): {tuple(a): 0.0 for a in actions}  # Convert actions to tuples
                   for i in range(row_size) for j in range(col_size)}

    for episode in range(episodes):
        current_state = start
        current_action = choose_action(current_state, qvalues, epsilon)
        terminal = False

        while not terminal:
            next_state, reward, terminal = take_action(current_state, current_action)
            next_action = choose_action(next_state, qvalues, epsilon) if not terminal else None

            # SARSA update
            current_value = qvalues[current_state][current_action]
            next_value = qvalues[next_state][next_action] if next_action is not None else 0
            qvalues[current_state][current_action] += alpha * (reward + gamma * next_value - current_value)

            current_state, current_action = next_state, next_action

    return qvalues

In [62]:
def qlearning(episodes, epsilon, start, alpha, gamma, qvalues=None):
    if qvalues is None:
        qvalues = {(i, j): {a: 0.0 for a in actions}
                   for i in range(row_size) for j in range(col_size)}

    for episode in range(episodes):
        current_state = start
        terminal = False

        while not terminal:
            current_action = choose_action(current_state, qvalues, epsilon)
            next_state, reward, terminal = take_action(current_state, current_action)

            # Q-Learning update
            current_value = qvalues[current_state][current_action]
            next_value = max(qvalues[next_state].values()) if not terminal else 0
            qvalues[current_state][current_action] += alpha * (reward + gamma * next_value - current_value)

            current_state = next_state

    return qvalues

In [63]:
def generate_policy_path(start, qvalues):
    current_state = start
    policy_path = [current_state]
    terminal = False

    while not terminal:
        current_action = max(qvalues[current_state], key=qvalues[current_state].get)
        next_state, _, terminal = take_action(current_state, current_action)
        policy_path.append(next_state)
        current_state = next_state
        if current_state == goal:  
            break

    return policy_path

In [64]:
# Run SARSA to get Q-values
qs_s = sarsa(1000, epsilon, start, alpha)

# Generate policy path from the learned Q-values
states_s = generate_policy_path(start, qs_s)

print(states_s)
showRoute(states_s)

[(3, 0), (2, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (2, 20), (3, 20)]
--------------------------------------------
          R R R R R R R R R R             
--------------------------------------------
R R R R R R         O       R R R R R R R 
--------------------------------------------
R                   O                   R 
--------------------------------------------
S C C C C C C C C C C C C C C C C C C C G 
--------------------------------------------


In [45]:
# Run Q-learning to get Q-values
qs_q = qlearning(1000, epsilon, start, alpha, gamma)

# Generate policy path from the learned Q-values
states_q = generate_policy_path(start, qs_q)

print(states_q)
showRoute(states_q)

[(3, 0), (2, 0), (2, 1), (2, 2), (2, 3), (1, 3), (1, 4), (1, 5), (1, 6), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (1, 14), (1, 15), (1, 16), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (3, 20)]
--------------------------------------------
            R R R R R R R R R             
--------------------------------------------
      R R R R       O       R R R         
--------------------------------------------
R R R R             O           R R R R R 
--------------------------------------------
S C C C C C C C C C C C C C C C C C C C G 
--------------------------------------------
